# Present PINK Validation results

Using the outputs from the `run_experiments.py` script, this notebook attempts to collate the results and make a nice table for digest

In [1]:
import pandas as pd
import numpy as np
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
mode = '../Results/Segments'
mode = '../Results/Fraction'
mode = '../Results/initial_results'

files = glob.glob(f'{mode}/*/*json')
dfs   = [pd.read_json(i) for i in files] 
print(len(dfs))

338


In [5]:
df = pd.concat(dfs)
df = df.reset_index(drop=True)

In [6]:
# Removes lists from the data, as this can kill the pivot

def transform(x):
    if isinstance(x, str):
        return x
    if isinstance(x, bool):
        return str(x)
    else:
        return '_'.join([str(i) for i in x])

df['log'] = df['log'].apply(transform)
df.rename(columns={'som-width':'width', 'som-height':'height',
                    '1_1_accuracy':'1_1', '1_2_accuracy':'1_2', '1_3_accuracy':'1_3',
                    '2_2_accuracy':'2_2', '2_3_accuracy':'2_3', 
                    '3_3_accuracy':'3_3',
                    'total_accuracy':'total'}, inplace=True)
df['WISE'] = df['validate_path'].str.contains('WISE_W1')

In [7]:
mask = df['WISE'] == False
mdf = pd.pivot_table(df[mask], index=['WISE' ,'norm','sigma','log','width','height', 'convex'],
                     values=['total', '1_1', '1_2', '1_3', '2_2', '2_3', '3_3'],
                     fill_value=0)

mdf.to_latex('Tables/No_WISE_pivot.tex', multirow=True, bold_rows=True)
mdf

1_1  1_2  1_3  2_2  2_3  \
WISE  norm  sigma log        width height convex                            
False False 0     False      3     3      False  0.43 0.25 0.31 0.00 0.20   
                             7     7      False  0.60 0.26 0.31 0.16 0.30   
                             10    10     False  0.59 0.27 0.29 0.23 0.29   
                             12    12     False  0.62 0.27 0.30 0.22 0.25   
      True  0     False      3     3      False  0.49 0.24 0.29 0.13 0.13   
                             7     7      False  0.61 0.29 0.28 0.18 0.19   
                             10    10     False  0.62 0.30 0.30 0.20 0.23   
                             12    12     False  0.64 0.30 0.30 0.18 0.22   
            3     False      3     3      False  0.55 0.23 0.27 0.18 0.16   
                             7     7      False  0.64 0.27 0.27 0.20 0.21   
                             10    10     False  0.67 0.26 0.30 0.22 0.22   
                             12    12     False  0.69 0.26 0.33 0.22 0.22   
                  True_False 3     3      False  0.57 0.24 0.32 0.26 0.19   
                             7     7      False  0.67 0.26 0.29 0.26 0.22   
                             10    10     False  0.67 0.29 0.37 0.25 0.26   
                             12    12     False  0.68 0.30 0.38 0.26 0.24   

                                                  3_3  total  
WISE  norm  sigma log        width height convex              
False False 0     False      3     3      False  0.00   0.41  
                             7     7      False  0.27   0.46  
                             10    10     False  0.34   0.47  
                             12    12     False  0.33   0.46  
      True  0     False      3     3      False  0.24   0.42  
                             7     7      False  0.28   0.46  
                             10    10     False  0.29   0.46  
                             12    12     False  0.28   0.47  
            3     False      3     3      False  0.22   0.43  
                             7     7      False  0.31   0.46  
                             10    10     False  0.29   0.47  
                             12    12     False  0.28   0.47  
                  True_False 3     3      False  0.27   0.46  
                             7     7      False  0.30   0.47  
                             10    10     False  0.31   0.49  
                             12    12     False  0.31   0.49

In [8]:
mdf = pd.pivot_table(df[~mask], index=['WISE' ,'norm','sigma','log','convex','width','height'],
                     values=['total', '1_1', '1_2', '1_3', '2_2', '2_3', '3_3'],
                     fill_value=0)

mdf.to_latex('Tables/WISE_pivot.tex', multirow=True, bold_rows=True)
mdf

1_1  1_2  1_3  2_2  2_3  3_3  \
WISE norm  sigma log        convex width height                                 
True False 0     False      False  3     3      0.38 0.00 0.00 0.00 0.00 0.00   
                                   7     7      0.39 0.17 0.09 0.00 0.00 0.40   
                                   10    10     0.38 0.05 0.14 0.00 0.12 0.11   
                                   12    12     0.38 0.06 0.09 0.17 0.13 0.11   
     True  0     False      False  3     3      0.39 0.00 0.20 0.00 0.00 0.00   
                                   7     7      0.46 0.29 0.31 0.15 0.16 0.25   
                                   10    10     0.48 0.27 0.31 0.14 0.17 0.31   
                                   12    12     0.48 0.31 0.30 0.19 0.22 0.28   
           3     False      False  3     3      0.45 0.21 0.27 0.00 0.00 0.00   
                                   7     7      0.46 0.24 0.31 0.19 0.16 0.26   
                                   10    10     0.48 0.28 0.32 0.24 0.23 0.29   
                                   12    12     0.48 0.24 0.31 0.22 0.17 0.26   
                 True_False False  3     3      0.48 0.19 0.29 0.00 0.00 0.00   
                                   7     7      0.52 0.25 0.29 0.22 0.17 0.32   
                                   10    10     0.53 0.26 0.30 0.21 0.20 0.30   
                                   12    12     0.53 0.28 0.33 0.25 0.19 0.30   
                            True   3     3      0.45 0.20 0.25 0.20 0.00 0.00   
                                   7     7      0.59 0.26 0.29 0.23 0.14 0.30   
                                   10    10     0.59 0.27 0.30 0.23 0.22 0.30   
                                   12    12     0.61 0.28 0.32 0.24 0.22 0.29   

                                                 total  
WISE norm  sigma log        convex width height         
True False 0     False      False  3     3        0.38  
                                   7     7        0.39  
                                   10    10       0.37  
                                   12    12       0.38  
     True  0     False      False  3     3        0.39  
                                   7     7        0.42  
                                   10    10       0.43  
                                   12    12       0.43  
           3     False      False  3     3        0.41  
                                   7     7        0.42  
                                   10    10       0.43  
                                   12    12       0.43  
                 True_False False  3     3        0.42  
                                   7     7        0.44  
                                   10    10       0.44  
                                   12    12       0.45  
                            True   3     3        0.41  
                                   7     7        0.45  
                                   10    10       0.46  
                                   12    12       0.46

In [9]:
mdf = pd.pivot_table(df, index=['WISE' ,'norm','sigma','log','convex','width','height'],
                     values=['total', '1_1', '1_2', '1_3', '2_2', '2_3', '3_3'],
                     fill_value=0)

mdf.to_latex('Tables/Complete_pivot.tex', multirow=True, bold_rows=True)
mdf

1_1  1_2  1_3  2_2  2_3  \
WISE  norm  sigma log        convex width height                            
False False 0     False      False  3     3      0.43 0.25 0.31 0.00 0.20   
                                    7     7      0.60 0.26 0.31 0.16 0.30   
                                    10    10     0.59 0.27 0.29 0.23 0.29   
                                    12    12     0.62 0.27 0.30 0.22 0.25   
      True  0     False      False  3     3      0.49 0.24 0.29 0.13 0.13   
                                    7     7      0.61 0.29 0.28 0.18 0.19   
                                    10    10     0.62 0.30 0.30 0.20 0.23   
                                    12    12     0.64 0.30 0.30 0.18 0.22   
            3     False      False  3     3      0.55 0.23 0.27 0.18 0.16   
                                    7     7      0.64 0.27 0.27 0.20 0.21   
                                    10    10     0.67 0.26 0.30 0.22 0.22   
                                    12    12     0.69 0.26 0.33 0.22 0.22   
                  True_False False  3     3      0.57 0.24 0.32 0.26 0.19   
                                    7     7      0.67 0.26 0.29 0.26 0.22   
                                    10    10     0.67 0.29 0.37 0.25 0.26   
                                    12    12     0.68 0.30 0.38 0.26 0.24   
True  False 0     False      False  3     3      0.38 0.00 0.00 0.00 0.00   
                                    7     7      0.39 0.17 0.09 0.00 0.00   
                                    10    10     0.38 0.05 0.14 0.00 0.12   
                                    12    12     0.38 0.06 0.09 0.17 0.13   
      True  0     False      False  3     3      0.39 0.00 0.20 0.00 0.00   
                                    7     7      0.46 0.29 0.31 0.15 0.16   
                                    10    10     0.48 0.27 0.31 0.14 0.17   
                                    12    12     0.48 0.31 0.30 0.19 0.22   
            3     False      False  3     3      0.45 0.21 0.27 0.00 0.00   
                                    7     7      0.46 0.24 0.31 0.19 0.16   
                                    10    10     0.48 0.28 0.32 0.24 0.23   
                                    12    12     0.48 0.24 0.31 0.22 0.17   
                  True_False False  3     3      0.48 0.19 0.29 0.00 0.00   
                                    7     7      0.52 0.25 0.29 0.22 0.17   
                                    10    10     0.53 0.26 0.30 0.21 0.20   
                                    12    12     0.53 0.28 0.33 0.25 0.19   
                             True   3     3      0.45 0.20 0.25 0.20 0.00   
                                    7     7      0.59 0.26 0.29 0.23 0.14   
                                    10    10     0.59 0.27 0.30 0.23 0.22   
                                    12    12     0.61 0.28 0.32 0.24 0.22   

                                                  3_3  total  
WISE  norm  sigma log        convex width height              
False False 0     False      False  3     3      0.00   0.41  
                                    7     7      0.27   0.46  
                                    10    10     0.34   0.47  
                                    12    12     0.33   0.46  
      True  0     False      False  3     3      0.24   0.42  
                                    7     7      0.28   0.46  
                                    10    10     0.29   0.46  
                                    12    12     0.28   0.47  
            3     False      False  3     3      0.22   0.43  
                                    7     7      0.31   0.46  
                                    10    10     0.29   0.47  
                                    12    12     0.28   0.47  
                  True_False False  3     3      0.27   0.46  
                                    7     7      0.30   0.47  
                                    10    10     0.31   0.49  
                                    12    12     0.3

In [10]:
df.columns.tolist()

['1_1',
 '1_1_correct',
 '1_1_wrong',
 '1_2',
 '1_2_correct',
 '1_2_wrong',
 '1_3',
 '1_3_correct',
 '1_3_wrong',
 '2_2',
 '2_2_correct',
 '2_2_wrong',
 '2_3',
 '2_3_correct',
 '2_3_wrong',
 '3_3',
 '3_3_correct',
 '3_3_wrong',
 'convex',
 'empty_neuron_attempts',
 'experiment',
 'log',
 'norm',
 'num-iter',
 'realisations',
 'sigma',
 'height',
 'width',
 'total',
 'total_correct',
 'total_wrong',
 'train_SOM_hash',
 'train_segment',
 'train_src_hash',
 'trained_SOM',
 'valid_src_hash',
 'validate_path',
 'weighted',
 'WISE']

In [16]:
def reduce_funct(v):
    return f'{np.mean(v):.2f}, {np.std(v):.2f}'
    return f'{np.mean(v):.2f}, {np.min(v):.2f}, {np.max(v):.2f}'
    return (np.mean(v), np.min(v), np.max(v))

mdf = pd.pivot_table(df, index=['WISE' ,'norm','sigma','log','convex','width','height'],
                     values=['total', '1_1', '1_2', '1_3', '2_2', '2_3', '3_3'],
                     fill_value=0,
                    aggfunc=reduce_funct)
mdf

1_1         1_2  \
WISE  norm  sigma log        convex width height                           
False False 0     False      False  3     3       0.43, 0.02  0.25, 0.07   
                                    7     7       0.60, 0.03  0.26, 0.02   
                                    10    10      0.59, 0.02  0.27, 0.02   
                                    12    12      0.62, 0.02  0.27, 0.02   
      True  0     False      False  3     3       0.49, 0.04  0.24, 0.02   
                                    7     7       0.61, 0.02  0.29, 0.04   
                                    10    10      0.62, 0.02  0.30, 0.02   
                                    12    12      0.64, 0.01  0.30, 0.02   
            3     False      False  3     3       0.55, 0.03  0.23, 0.02   
                                    7     7       0.64, 0.02  0.27, 0.02   
                                    10    10      0.67, 0.01  0.26, 0.02   
                                    12    12      0.69, 0.01  0.26, 0.02   
                  True_False False  3     3       0.57, 0.05  0.24, 0.04   
                                    7     7       0.67, 0.02  0.26, 0.02   
                                    10    10      0.67, 0.02  0.29, 0.02   
                                    12    12      0.68, 0.01  0.30, 0.02   
True  False 0     False      False  3     3       0.38, 0.00    nan, nan   
                                    7     7       0.39, 0.01  0.17, 0.00   
                                    10    10      0.38, 0.01  0.05, 0.07   
                                    12    12      0.38, 0.01  0.06, 0.07   
      True  0     False      False  3     3       0.39, 0.02    nan, nan   
                                    7     7       0.46, 0.02  0.29, 0.04   
                                    10    10      0.48, 0.01  0.27, 0.03   
                                    12    12      0.48, 0.02  0.31, 0.05   
            3     False      False  3     3       0.45, 0.03  0.21, 0.00   
                                    7     7       0.46, 0.01  0.24, 0.04   
                                    10    10      0.48, 0.01  0.28, 0.05   
                                    12    12      0.48, 0.01  0.24, 0.05   
                  True_False False  3     3       0.48, 0.02  0.19, 0.03   
                                    7     7       0.52, 0.01  0.25, 0.03   
                                    10    10      0.53, 0.01  0.26, 0.03   
                                    12    12      0.53, 0.01  0.28, 0.03   
                             True   3     3       0.45, 0.03  0.20, 0.02   
                                    7     7       0.59, 0.01  0.26, 0.02   
                                    10    10      0.59, 0.02  0.27, 0.03   
                                    12    12      0.61, 0.01  0.28, 0.02   

                                                         1_3         2_2  \
WISE  norm  sigma log        convex width height                           
False False 0     False      False  3     3       0.31, 0.03    nan, nan   
                                    7     7       0.31, 0.02  0.16, 0.02   
                                    10    10      0.29, 0.02  0.23, 0.05   
                                    12    12      0.30, 0.02  0.22, 0.04   
      True  0     False      False  3     3       0.29, 0.04  0.13, 0.02   
                                    7     7       0.28, 0.03  0.18, 0.04   
                                    10    10      0.30, 0.02  0.20, 0.05   
                                    12    12      0.30, 0.02  0.18, 0.02   
            3     False      False  3     3       0.27, 0.01  0.18, 0.04   
                                    7     7       0.27, 0.02  0.20, 0.04   
                                    10    10      0.30, 0.02  0.22, 0.02   
                                    12    12      0.33, 0.03  0.22, 0.02   
                  True_False False  3     3       0.32, 0.04  0.26, 0.09   
                            

In [15]:
def reduce_funct(v):
    return len(v)
    return (np.mean(v), np.min(v), np.max(v))

mdf = pd.pivot_table(df, index=['WISE' ,'norm','sigma','log','convex','width','height'],
                     values=['total', '1_1', '1_2', '1_3', '2_2', '2_3', '3_3'],
                     fill_value=0,
                    aggfunc=reduce_funct)
mdf

1_1  1_2  1_3  2_2  2_3  \
WISE  norm  sigma log        convex width height                            
False False 0     False      False  3     3        10   10   10   10   10   
                                    7     7        10   10   10   10   10   
                                    10    10       10   10   10   10   10   
                                    12    12       10   10   10   10   10   
      True  0     False      False  3     3        10   10   10   10   10   
                                    7     7        10   10   10   10   10   
                                    10    10       10   10   10   10   10   
                                    12    12       10   10   10   10   10   
            3     False      False  3     3        10   10   10   10   10   
                                    7     7        10   10   10   10   10   
                                    10    10       10   10   10   10   10   
                                    12    12       10   10   10   10   10   
                  True_False False  3     3        10   10   10   10   10   
                                    7     7        10   10   10   10   10   
                                    10    10       10   10   10   10   10   
                                    12    12       10   10   10   10   10   
True  False 0     False      False  3     3         9    9    9    9    9   
                                    7     7         9    9    9    9    9   
                                    10    10        9    9    9    9    9   
                                    12    12        9    9    9    9    9   
      True  0     False      False  3     3         9    9    9    9    9   
                                    7     7         9    9    9    9    9   
                                    10    10        9    9    9    9    9   
                                    12    12        9    9    9    9    9   
            3     False      False  3     3         9    9    9    9    9   
                                    7     7         9    9    9    9    9   
                                    10    10        9    9    9    9    9   
                                    12    12        9    9    9    9    9   
                  True_False False  3     3         9    9    9    9    9   
                                    7     7         9    9    9    9    9   
                                    10    10        9    9    9    9    9   
                                    12    12        9    9    9    9    9   
                             True   3     3         9    9    9    9    9   
                                    7     7         9    9    9    9    9   
                                    10    10        8    8    8    8    8   
                                    12    12        8    8    8    8    8   

                                                  3_3  total  
WISE  norm  sigma log        convex width height              
False False 0     False      False  3     3        10     10  
                                    7     7        10     10  
                                    10    10       10     10  
                                    12    12       10     10  
      True  0     False      False  3     3        10     10  
                                    7     7        10     10  
                                    10    10       10     10  
                                    12    12       10     10  
            3     False      False  3     3        10     10  
                                    7     7        10     10  
                                    10    10       10     10  
                                    12    12       10     10  
                  True_False False  3     3        10     10  
                                    7     7        10     10  
                                    10    10       10     10  
                                    12    12       1

In [13]:
df

,1_1,1_1_correct,1_1_wrong,1_2,1_2_correct,1_2_wrong,1_3,1_3_correct,1_3_wrong,2_2,...,total_correct,total_wrong,train_SOM_hash,train_segment,train_src_hash,trained_SOM,valid_src_hash,validate_path,weighted,WISE
0,0.44,1149,1457,0.22,41.00,145.00,0.34,132.00,251.00,nan,...,1343,1924,fc9ca33e10232ded78e725558021d068e51aaeff,0,0d9d8c13cda838daae09af90a597e66e271f8053,Script_Experiments_Fractions_Trials/FIRST_WISE...,9437ad9566ce7a4f468225b9e070cbb6510aadca,Script_Experiments_Fractions_Trials/FIRST_WISE...,False,True
1,0.64,1104,619,0.35,102.00,190.00,0.30,168.00,390.00,0.14,...,1533,1732,fd601e5bcb46abc8d4033e71afba0db508d5899c,0,2333375e1fd809712fbbee7bd6503f116d15a5d1,Script_Experiments_Fractions_Trials/FIRST_Norm...,f6ffc205646b502d2a2891e08cdf89fe0952ced7,Script_Experiments_Fractions_Trials/FIRST_Norm...,False,False
2,0.45,1215,1477,0.22,28.00,100.00,0.30,118.00,278.00,nan,...,1369,1898,a52e49e60c6abf4511b438e73c3028ef78fb7811,0,c7b411bc2f1e0bed7d1a7a5963a7f4f8ced54a7e,Script_Experiments_Fractions_Trials/FIRST_WISE...,b849725f10d176c8a1a2de0ca11f1dfc21ba4a02,Script_Experiments_Fractions_Trials/FIRST_WISE...,False,True
3,0.66,1097,559,0.29,126.00,310.00,0.32,148.00,318.00,0.18,...,1536,1731,b391da5e36c22b6e084c572b3300d139e023c2e9,0,eba37f5ba892226b2b7ee067788efb63630781ec,Script_Experiments_Fractions_Trials/FIRST_Norm...,ab7baa562d31b3b417c64ad9880325b2d56fbb06,Script_Experiments_Fractions_Trials/FIRST_Norm...,False,False
4,0.45,1127,1379,0.22,35.00,127.00,0.24,142.00,457.00,nan,...,1304,1963,94baed8abcd9d0a338420d78e8d8933faad0cfb9,0,c912862f64d2e459a75967317e400faded3bf831,Script_Experiments_Fractions_Trials/FIRST_WISE...,f2020b0302e422b36e969bc837cb95accc4179d8,Script_Experiments_Fractions_Trials/FIRST_WISE...,False,True
5,0.43,1199,1605,nan,nan,nan,0.33,155.00,308.00,nan,...,1354,1913,43af9e9c4714e43d9e22f0686ca404af42230bf6,0,1c1bff7af33a3993f2d8796c1f4d01d42f404f8f,Script_Experiments_Fractions_Trials/FIRST_NoNo...,bdbf11cf87e1c5ca777611f67d7bef80bbdca110,Script_Experiments_Fractions_Trials/FIRST_NoNo...,False,False
6,0.48,1146,1248,0.21,86.00,318.00,0.34,158.00,311.00,nan,...,1390,1877,b44a1e7805787d62c55379f7c8ace01d3fe9530f,0,1e69ea595d1cbba174d7232954cfb9ac886cf03f,Script_Experiments_Fractions_Trials/FIRST_WISE...,36ce9b7c554109bc98b3082fcd5f1315dbd1a69d,Script_Experiments_Fractions_Trials/FIRST_WISE...,False,True
7,0.50,1169,1187,nan,nan,nan,0.23,208.00,703.00,nan,...,1377,1890,e08c492faf7b6db7235a7eafd302efbc7a0a1bc0,0,1424e6aea93c4ca2aac3af5158df4fe17af081f8,Script_Experiments_Fractions_Trials/FIRST_WISE...,96074f36cafb836a1b068ccc7db31ef5079a3928,Script_Experiments_Fractions_Trials/FIRST_WISE...,False,True
8,0.46,1164,1353,0.17,11.00,52.00,0.27,185.00,502.00,nan,...,1360,1907,4bc30565c681cdc9ec06577bc0a2a92c880304c2,0,50fbda1e8682e2d17c1e0e46c65b74e92ee6dd3e,Script_Experiments_Fractions_Trials/FIRST_NoNo...,b0c77f7f745d656a9eb523c1cbb3cac183a596fc,Script_Experiments_Fractions_Trials/FIRST_NoNo...,False,False
9,0.42,1161,1577,0.28,28.00,72.00,0.33,115.00,236.00,0.19,...,1317,1950,9618f3d1681aec60de3e6c8b6b69ed01387fc00c,0,459e27704e7a045a414e142e58d4553323de7be4,Script_Experiments_Fractions_Trials/FIRST_WISE...,6db8c1c88ef2ee386b2b1fa2426800a64316fb6e,Script_Experiments_Fractions_Trials/FIRST_WISE...,False,True


In [86]:
mode = '../Results/initial_results'
files = glob.glob(f'{mode}/*/*json')
dfs   = [pd.read_json(i) for i in files] 